# Tabnet 2 - Modelo luego del primer año

In [36]:
!pip install pytorch-tabnet
!pip install tabnet
!pip install wget

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

In [37]:
df=pd.read_csv("C:/Users/Profesor/data_imputation_paramodel2_train.txt",header=0, sep=",",na_values='\t', encoding='latin-1') #train
df_valid=pd.read_csv("C:/Users/Profesor/data_imputation_paramodel2_valid.txt",header=0, sep=",",na_values='\t', encoding='latin-1')
df_test=pd.read_csv("C:/Users/Profesor/data_imputation_paramodel2_test.txt",header=0, sep=",",na_values='\t', encoding='latin-1')
#df['GRAD'] = df.GRAD.map(dict(Y=1, N=0))


In [38]:
#Agregar una nueva col con el set al que pertenecen 
df["Set"]="train"
df_valid["Set"]="valid"
df_test["Set"]="test"

In [39]:
#Concatenando
df=df.append(df_valid)
df=df.append(df_test)

In [40]:
target = 'GRAD'
#if "Set" not in df.columns:
#    df["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(df.shape[0],))

train_indices = df[df.Set=="train"].index
valid_indices = df[df.Set=="valid"].index
test_indices = df[df.Set=="test"].index

In [41]:
valid_indices

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855],
           dtype='int64', length=1856)

In [43]:
df.iloc[21402,]

GRAD                                             N
Year                                          2004
Faculty             Facultad de Ciencias Agrícolas
Major                                         TMAG
Apt_Verbal                                     559
Aprov_Matem                                    478
Apt_Matem                                      551
Aprov_Espanol                                  459
Aprov_Ingles                                   465
INGRESO_FAMILIAR                         I4_B30A50
EDUC_PADRE                            Bachillerato
EDUC_MADRE                            Bachillerato
Gender                                           M
School_Type                                 Public
GPA_1ER_ANO                                   2.17
Rel_Stud_GPA                              0.989497
Rel_School_GPA                            0.718531
Set                                           test
Name: 0, dtype: object

In [44]:
df

,GRAD,Year,Faculty,Major,Apt_Verbal,Aprov_Matem,Apt_Matem,Aprov_Espanol,Aprov_Ingles,INGRESO_FAMILIAR,EDUC_PADRE,EDUC_MADRE,Gender,School_Type,GPA_1ER_ANO,Rel_Stud_GPA,Rel_School_GPA,Set
0,N,2000,Facultad de Artes y Ciencias - Artes,EFEN,530,444,447,453,389,I1_L12.5,HS,HS,M,Private,2.11,0.996736,0.791875,train
1,N,2000,Facultad de Artes y Ciencias - Ciencias,CIFI,544,630,594,500,369,I1_L12.5,LHS,HS,F,Public,2.51,0.880744,0.692077,train
2,N,2000,Facultad de Ciencias Agrícolas,INPE,598,615,711,590,666,I5_O50,Bachillerato,Grad,M,Public,1.78,1.013281,0.782722,train
3,Y,2000,Facultad de Ingeniería,ICOM,680,786,668,574,573,I1_L12.5,Bachillerato,LHS,F,Public,3.15,1.025103,0.688084,train
4,N,2000,Facultad de Administración de Empresas,MERC,593,728,695,593,602,I1_L12.5,Bachillerato,LHS,F,Private,2.77,1.089578,0.694022,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,N,2000,Facultad de Ingeniería,INEL,603,670,706,550,666,I4_B30A50,Bachillerato,Grad,M,Private,2.66,1.032747,0.819743,test
1834,N,2010,Facultad de Ingeniería,INQU,622,662,761,760,669,I1_L12.5,Assoc or Less,Assoc or Less,M,Private,2.03,1.062871,0.803084,test
1835,N,2008,Facultad de Artes y Ciencias - Artes,PSIC,534,488,490,516,474,I2_B12.5A20,HS,Bachillerato,F,Public,2.51,0.887323,0.666503,test
1836,N,2010,Facultad de Ciencias Agrícolas,AGNE,477,568,542,400,650,I1_L12.5,Grad,Bachillerato,M,Private,1.00,1.031935,0.653546,test


In [45]:
nunique = df.nunique()
types = df.dtypes

categorical_columns = []
categorical_dims =  {}
for col in df.columns:
    if types[col] == 'object' or nunique[col] < 200:  #The nunique() function is used to count distinct observations over requested axis.
        print(col, df[col].nunique())
        l_enc = LabelEncoder()
        df[col] = df[col].fillna("VV_likely") 
        df[col] = l_enc.fit_transform(df[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        df.fillna(df.loc[train_indices, col].mean(), inplace=True)

GRAD 2
Year 12
Faculty 5
Major 54
INGRESO_FAMILIAR 5
EDUC_PADRE 6
EDUC_MADRE 6
Gender 2
School_Type 3
Set 3


In [46]:
df

,GRAD,Year,Faculty,Major,Apt_Verbal,Aprov_Matem,Apt_Matem,Aprov_Espanol,Aprov_Ingles,INGRESO_FAMILIAR,EDUC_PADRE,EDUC_MADRE,Gender,School_Type,GPA_1ER_ANO,Rel_Stud_GPA,Rel_School_GPA,Set
0,0,1,1,19,530,444,447,453,389,0,3,3,1,1,2.11,0.996736,0.791875,1
1,0,1,2,10,544,630,594,500,369,0,4,3,0,2,2.51,0.880744,0.692077,1
2,0,1,3,39,598,615,711,590,666,4,1,2,1,2,1.78,1.013281,0.782722,1
3,1,1,4,33,680,786,668,574,573,0,1,4,0,2,3.15,1.025103,0.688084,1
4,0,1,0,44,593,728,695,593,602,0,1,4,0,1,2.77,1.089578,0.694022,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,0,1,4,35,603,670,706,550,666,3,1,2,1,1,2.66,1.032747,0.819743,0
1834,0,11,4,40,622,662,761,760,669,0,0,0,1,1,2.03,1.062871,0.803084,0
1835,0,9,1,47,534,488,490,516,474,1,3,1,0,2,2.51,0.887323,0.666503,0
1836,0,11,3,2,477,568,542,400,650,0,2,1,1,1,1.00,1.031935,0.653546,0


In [47]:
unused_feat = ['Set']

features = [ col for col in df.columns if col not in unused_feat+[target]] #Select features without target and Set.

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns] #Select features without target and Set in "features" is categorical.

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns] #Select features without target and Set in "features" is categorical with count distinct observations.

In [48]:
df[features] = df[features].apply(lambda x: 1/(1+ np.exp(-x)), axis=0)  #Softmax Normalization

X_train = df[features].values[train_indices]
y_train = df[target].values[train_indices]

X_valid = df[features].values[valid_indices]
y_valid = df[target].values[valid_indices]

X_test = df[features].values[test_indices]
y_test = df[target].values[test_indices]

# Tuning 

In [49]:
#hyperparameters

#optimizer_params_list=dict(lr=[2e-2,1])

max_epochs_list = [20,21,22,23,24,25,26,27,28,29,30]
batch_size_list = [1024, 1025, 1026,1027,1028,1029,1030]
virtual_batch_size_list = [128, 129, 130, 131, 132,133, 134]
patience_list = [20,21,22,23,24,25,26]

In [50]:
def tabnet_grid_search(max_epochs,batch_size,virtual_batch_size,patience):
  model = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=1
                      )
  model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=max_epochs , patience=patience,
    batch_size=batch_size, virtual_batch_size=virtual_batch_size,
    num_workers=0,
    weights=1,
    drop_last=False
    )
  Report = classification_report(y_test,model.predict(X_test),labels=[0,1], output_dict=True)
  return ([max_epochs,batch_size,virtual_batch_size,patience,Report['0']['recall']])

In [51]:
#Loop
result_list = []

for max_epochs in max_epochs_list:
  for batch_size in batch_size_list:
    for virtual_batch_size in virtual_batch_size_list:
      for patience in patience_list:
        result_list.append(tabnet_grid_search(max_epochs,batch_size,virtual_batch_size,patience))

Result_tabnet = pd.DataFrame(result_list, columns=['max_epochs','batch_size','virtual_batch_size','patience', 'Recall'])

Device used : cpu
epoch 0  | loss: 0.66855 | train_auc: 0.5151  | valid_auc: 0.52258 |  0:00:01s
epoch 1  | loss: 0.56129 | train_auc: 0.63101 | valid_auc: 0.64026 |  0:00:03s
epoch 2  | loss: 0.53619 | train_auc: 0.70169 | valid_auc: 0.71088 |  0:00:05s
epoch 3  | loss: 0.53399 | train_auc: 0.78858 | valid_auc: 0.79446 |  0:00:07s
epoch 4  | loss: 0.52879 | train_auc: 0.70129 | valid_auc: 0.70001 |  0:00:08s
epoch 5  | loss: 0.53054 | train_auc: 0.48984 | valid_auc: 0.45881 |  0:00:10s
epoch 6  | loss: 0.52753 | train_auc: 0.48424 | valid_auc: 0.46321 |  0:00:12s
epoch 7  | loss: 0.52885 | train_auc: 0.8092  | valid_auc: 0.82027 |  0:00:13s
epoch 8  | loss: 0.53422 | train_auc: 0.79553 | valid_auc: 0.80891 |  0:00:15s
epoch 9  | loss: 0.52796 | train_auc: 0.79146 | valid_auc: 0.80333 |  0:00:17s
epoch 10 | loss: 0.53055 | train_auc: 0.79893 | valid_auc: 0.80999 |  0:00:19s
epoch 11 | loss: 0.52658 | train_auc: 0.80499 | valid_auc: 0.81528 |  0:00:20s
epoch 12 | loss: 0.53037 | train_a

epoch 15 | loss: 0.53148 | train_auc: 0.80832 | valid_auc: 0.81887 |  0:00:30s
epoch 16 | loss: 0.53042 | train_auc: 0.80947 | valid_auc: 0.82075 |  0:00:32s
epoch 17 | loss: 0.52375 | train_auc: 0.80816 | valid_auc: 0.81891 |  0:00:34s
epoch 18 | loss: 0.53323 | train_auc: 0.80891 | valid_auc: 0.82002 |  0:00:36s
epoch 19 | loss: 0.52662 | train_auc: 0.80899 | valid_auc: 0.81933 |  0:00:38s
Stop training because you reached max_epochs = 20 with best_epoch = 16 and best_valid_auc = 0.82075
Best weights from best epoch are automatically used!
Device used : cpu
epoch 0  | loss: 0.66855 | train_auc: 0.5151  | valid_auc: 0.52258 |  0:00:01s
epoch 1  | loss: 0.56129 | train_auc: 0.63101 | valid_auc: 0.64026 |  0:00:03s
epoch 2  | loss: 0.53619 | train_auc: 0.70169 | valid_auc: 0.71088 |  0:00:05s
epoch 3  | loss: 0.53399 | train_auc: 0.78858 | valid_auc: 0.79446 |  0:00:07s
epoch 4  | loss: 0.52879 | train_auc: 0.70129 | valid_auc: 0.70001 |  0:00:09s
epoch 5  | loss: 0.53054 | train_auc: 0

epoch 8  | loss: 0.53422 | train_auc: 0.79553 | valid_auc: 0.80891 |  0:00:17s
epoch 9  | loss: 0.52796 | train_auc: 0.79146 | valid_auc: 0.80333 |  0:00:19s
epoch 10 | loss: 0.53055 | train_auc: 0.79893 | valid_auc: 0.80999 |  0:00:21s
epoch 11 | loss: 0.52658 | train_auc: 0.80499 | valid_auc: 0.81528 |  0:00:23s
epoch 12 | loss: 0.53037 | train_auc: 0.80212 | valid_auc: 0.81273 |  0:00:25s
epoch 13 | loss: 0.52469 | train_auc: 0.80746 | valid_auc: 0.81733 |  0:00:27s
epoch 14 | loss: 0.53245 | train_auc: 0.80923 | valid_auc: 0.82041 |  0:00:29s
epoch 15 | loss: 0.53148 | train_auc: 0.80832 | valid_auc: 0.81887 |  0:00:31s
epoch 16 | loss: 0.53042 | train_auc: 0.80947 | valid_auc: 0.82075 |  0:00:33s
epoch 17 | loss: 0.52375 | train_auc: 0.80816 | valid_auc: 0.81891 |  0:00:35s
epoch 18 | loss: 0.53323 | train_auc: 0.80891 | valid_auc: 0.82002 |  0:00:37s
epoch 19 | loss: 0.52662 | train_auc: 0.80899 | valid_auc: 0.81933 |  0:00:39s
Stop training because you reached max_epochs = 20 wi

epoch 1  | loss: 0.56129 | train_auc: 0.63101 | valid_auc: 0.64026 |  0:00:04s
epoch 2  | loss: 0.53619 | train_auc: 0.70169 | valid_auc: 0.71088 |  0:00:06s
epoch 3  | loss: 0.53399 | train_auc: 0.78858 | valid_auc: 0.79446 |  0:00:08s
epoch 4  | loss: 0.52879 | train_auc: 0.70129 | valid_auc: 0.70001 |  0:00:09s
epoch 5  | loss: 0.53054 | train_auc: 0.48984 | valid_auc: 0.45881 |  0:00:11s
epoch 6  | loss: 0.52753 | train_auc: 0.48424 | valid_auc: 0.46321 |  0:00:13s
epoch 7  | loss: 0.52885 | train_auc: 0.8092  | valid_auc: 0.82027 |  0:00:15s
epoch 8  | loss: 0.53422 | train_auc: 0.79553 | valid_auc: 0.80891 |  0:00:17s
epoch 9  | loss: 0.52796 | train_auc: 0.79146 | valid_auc: 0.80333 |  0:00:19s
epoch 10 | loss: 0.53055 | train_auc: 0.79893 | valid_auc: 0.80999 |  0:00:22s
epoch 11 | loss: 0.52658 | train_auc: 0.80499 | valid_auc: 0.81528 |  0:00:24s
epoch 12 | loss: 0.53037 | train_auc: 0.80212 | valid_auc: 0.81273 |  0:00:25s
epoch 13 | loss: 0.52469 | train_auc: 0.80746 | vali

epoch 17 | loss: 0.52375 | train_auc: 0.80816 | valid_auc: 0.81891 |  0:00:39s
epoch 18 | loss: 0.53323 | train_auc: 0.80891 | valid_auc: 0.82002 |  0:00:41s
epoch 19 | loss: 0.52662 | train_auc: 0.80899 | valid_auc: 0.81933 |  0:00:43s
Stop training because you reached max_epochs = 20 with best_epoch = 16 and best_valid_auc = 0.82075
Best weights from best epoch are automatically used!
Device used : cpu
epoch 0  | loss: 0.66855 | train_auc: 0.5151  | valid_auc: 0.52258 |  0:00:02s
epoch 1  | loss: 0.56129 | train_auc: 0.63101 | valid_auc: 0.64026 |  0:00:04s
epoch 2  | loss: 0.53619 | train_auc: 0.70169 | valid_auc: 0.71088 |  0:00:06s
epoch 3  | loss: 0.53399 | train_auc: 0.78858 | valid_auc: 0.79446 |  0:00:08s
epoch 4  | loss: 0.52879 | train_auc: 0.70129 | valid_auc: 0.70001 |  0:00:10s
epoch 5  | loss: 0.53054 | train_auc: 0.48984 | valid_auc: 0.45881 |  0:00:13s
epoch 6  | loss: 0.52753 | train_auc: 0.48424 | valid_auc: 0.46321 |  0:00:15s
epoch 7  | loss: 0.52885 | train_auc: 0

...

epoch 17 | loss: 0.52291 | train_auc: 0.80871 | valid_auc: 0.81903 |  0:00:38s
epoch 18 | loss: 0.53245 | train_auc: 0.80635 | valid_auc: 0.81683 |  0:00:40s
epoch 19 | loss: 0.52768 | train_auc: 0.80887 | valid_auc: 0.82038 |  0:00:42s
Stop training because you reached max_epochs = 20 with best_epoch = 12 and best_valid_auc = 0.82055
Best weights from best epoch are automatically used!
Device used : cpu
epoch 0  | loss: 0.63925 | train_auc: 0.32788 | valid_auc: 0.30028 |  0:00:02s
epoch 1  | loss: 0.54259 | train_auc: 0.30656 | valid_auc: 0.2939  |  0:00:04s
epoch 2  | loss: 0.53067 | train_auc: 0.8026  | valid_auc: 0.81403 |  0:00:06s
epoch 3  | loss: 0.53236 | train_auc: 0.80226 | valid_auc: 0.80681 |  0:00:08s
epoch 4  | loss: 0.52927 | train_auc: 0.79348 | valid_auc: 0.8055  |  0:00:10s
epoch 5  | loss: 0.53115 | train_auc: 0.772   | valid_auc: 0.78491 |  0:00:12s
epoch 6  | loss: 0.53163 | train_auc: 0.67589 | valid_auc: 0.68621 |  0:00:15s
epoch 7  | loss: 0.52878 | train_auc: 0

epoch 10 | loss: 0.53281 | train_auc: 0.79085 | valid_auc: 0.80601 |  0:00:24s
epoch 11 | loss: 0.52625 | train_auc: 0.80402 | valid_auc: 0.81717 |  0:00:26s
epoch 12 | loss: 0.53219 | train_auc: 0.80833 | valid_auc: 0.82055 |  0:00:28s
epoch 13 | loss: 0.52568 | train_auc: 0.79945 | valid_auc: 0.81416 |  0:00:31s
epoch 14 | loss: 0.53698 | train_auc: 0.80819 | valid_auc: 0.81871 |  0:00:33s
epoch 15 | loss: 0.53213 | train_auc: 0.80853 | valid_auc: 0.81783 |  0:00:35s
epoch 16 | loss: 0.52853 | train_auc: 0.80539 | valid_auc: 0.81446 |  0:00:37s
epoch 17 | loss: 0.52291 | train_auc: 0.80871 | valid_auc: 0.81903 |  0:00:39s
epoch 18 | loss: 0.53245 | train_auc: 0.80635 | valid_auc: 0.81683 |  0:00:41s
epoch 19 | loss: 0.52768 | train_auc: 0.80887 | valid_auc: 0.82038 |  0:00:44s
Stop training because you reached max_epochs = 20 with best_epoch = 12 and best_valid_auc = 0.82055
Best weights from best epoch are automatically used!
Device used : cpu
epoch 0  | loss: 0.63925 | train_auc: 0

epoch 3  | loss: 0.53236 | train_auc: 0.80226 | valid_auc: 0.80681 |  0:00:08s
epoch 4  | loss: 0.52927 | train_auc: 0.79348 | valid_auc: 0.8055  |  0:00:10s
epoch 5  | loss: 0.53115 | train_auc: 0.772   | valid_auc: 0.78491 |  0:00:11s
epoch 6  | loss: 0.53163 | train_auc: 0.67589 | valid_auc: 0.68621 |  0:00:13s
epoch 7  | loss: 0.52878 | train_auc: 0.77045 | valid_auc: 0.78016 |  0:00:16s
epoch 8  | loss: 0.53273 | train_auc: 0.74903 | valid_auc: 0.75641 |  0:00:18s
epoch 9  | loss: 0.52707 | train_auc: 0.7148  | valid_auc: 0.7231  |  0:00:20s
epoch 10 | loss: 0.53281 | train_auc: 0.79085 | valid_auc: 0.80601 |  0:00:22s
epoch 11 | loss: 0.52625 | train_auc: 0.80402 | valid_auc: 0.81717 |  0:00:24s
epoch 12 | loss: 0.53219 | train_auc: 0.80833 | valid_auc: 0.82055 |  0:00:26s
epoch 13 | loss: 0.52568 | train_auc: 0.79945 | valid_auc: 0.81416 |  0:00:27s
epoch 14 | loss: 0.53698 | train_auc: 0.80819 | valid_auc: 0.81871 |  0:00:29s
epoch 15 | loss: 0.53213 | train_auc: 0.80853 | vali

epoch 19 | loss: 0.52768 | train_auc: 0.80887 | valid_auc: 0.82038 |  0:00:44s
Stop training because you reached max_epochs = 20 with best_epoch = 12 and best_valid_auc = 0.82055
Best weights from best epoch are automatically used!
Device used : cpu
epoch 0  | loss: 0.63925 | train_auc: 0.32788 | valid_auc: 0.30028 |  0:00:01s
epoch 1  | loss: 0.54259 | train_auc: 0.30656 | valid_auc: 0.2939  |  0:00:03s
epoch 2  | loss: 0.53067 | train_auc: 0.8026  | valid_auc: 0.81403 |  0:00:05s
epoch 3  | loss: 0.53236 | train_auc: 0.80226 | valid_auc: 0.80681 |  0:00:07s
epoch 4  | loss: 0.52927 | train_auc: 0.79348 | valid_auc: 0.8055  |  0:00:09s
epoch 5  | loss: 0.53115 | train_auc: 0.772   | valid_auc: 0.78491 |  0:00:12s
epoch 6  | loss: 0.53163 | train_auc: 0.67589 | valid_auc: 0.68621 |  0:00:14s
epoch 7  | loss: 0.52878 | train_auc: 0.77045 | valid_auc: 0.78016 |  0:00:16s
epoch 8  | loss: 0.53273 | train_auc: 0.74903 | valid_auc: 0.75641 |  0:00:18s
epoch 9  | loss: 0.52707 | train_auc: 0

KeyboardInterrupt: 

In [52]:
Result_tabnet = pd.DataFrame(result_list, columns=['max_epochs','batch_size','virtual_batch_size','patience', 'Recall'])
Result_tabnet.sort_values(by=['Recall'], inplace=True, ascending=False)

In [53]:
Result_tabnet

,max_epochs,batch_size,virtual_batch_size,patience,Recall
64,20,1025,130,21,0.708543
66,20,1025,130,23,0.708543
74,20,1025,131,24,0.708543
73,20,1025,131,23,0.708543
72,20,1025,131,22,0.708543
...,...,...,...,...,...
108,20,1026,129,23,0.444724
107,20,1026,129,22,0.444724
106,20,1026,129,21,0.444724
105,20,1026,129,20,0.444724


# Modelo luego del primer año

In [56]:
clf = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=1
                      )

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=20 , patience=25,
    batch_size=1028, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
)

Device used : cpu
epoch 0  | loss: 0.62909 | train_auc: 0.59586 | valid_auc: 0.58236 |  0:00:01s
epoch 1  | loss: 0.54465 | train_auc: 0.33108 | valid_auc: 0.32773 |  0:00:03s
epoch 2  | loss: 0.52909 | train_auc: 0.58148 | valid_auc: 0.57513 |  0:00:05s
epoch 3  | loss: 0.53064 | train_auc: 0.7417  | valid_auc: 0.75544 |  0:00:06s
epoch 4  | loss: 0.53299 | train_auc: 0.72682 | valid_auc: 0.74154 |  0:00:08s
epoch 5  | loss: 0.53132 | train_auc: 0.79685 | valid_auc: 0.80604 |  0:00:10s
epoch 6  | loss: 0.52935 | train_auc: 0.74159 | valid_auc: 0.74413 |  0:00:11s
epoch 7  | loss: 0.52988 | train_auc: 0.77886 | valid_auc: 0.78679 |  0:00:13s
epoch 8  | loss: 0.53496 | train_auc: 0.79841 | valid_auc: 0.80939 |  0:00:15s
epoch 9  | loss: 0.53198 | train_auc: 0.78773 | valid_auc: 0.8017  |  0:00:17s
epoch 10 | loss: 0.53582 | train_auc: 0.80777 | valid_auc: 0.81904 |  0:00:18s
epoch 11 | loss: 0.5328  | train_auc: 0.8041  | valid_auc: 0.81599 |  0:00:20s
epoch 12 | loss: 0.53306 | train_a

In [57]:
Report = classification_report(y_test,clf.predict(X_test),labels=[0,1])
print('Classification report : \n',Report)

Classification report : 
               precision    recall  f1-score   support

           0       0.68      0.75      0.71       796
           1       0.79      0.73      0.76      1042

    accuracy                           0.74      1838
   macro avg       0.74      0.74      0.74      1838
weighted avg       0.74      0.74      0.74      1838



In [58]:
preds = clf.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)
print(f"AUC : {test_auc}")

AUC : 0.81954337426094


In [59]:
classification_report(y_test,clf.predict(X_test),labels=[0,1], output_dict=True)['0']['recall']

0.7462311557788944

In [60]:
classification_report(y_test,clf.predict(X_test),labels=[0,1], output_dict=True)['0']['f1-score']

0.7118034751348112

In [61]:
pd.crosstab(clf.predict(X_test), y_test, 
            rownames=['pred'], 
            colnames=['Actual'], margins=False, margins_name="Total")

Actual,0,1
pred,,
0,594,279
1,202,763


In [62]:
1-classification_report(y_test,clf.predict(X_test),labels=[0,1], output_dict=True)['0']['recall']

0.25376884422110557

In [63]:
202/(202+594)

0.2537688442211055

In [64]:
202/(202+594+279+763)

0.10990206746463548